# Lab Class 1

## Code from Demo 1.1:

In [1]:
#include <iostream>
#include <cmath>
using namespace std;

In [2]:

double normalDistribution_int(double x,int N)
{
  if(x<-10.)return 0.;
  if(x>10.)return 1.;
  // range of integration
  double a=0,b=x;
  // local variables
  double s,h,sum=0.;
  // inialise the variables
  h=(b-a)/N;
  // add in the first few terms
  sum = sum + exp(-a*a/2.) + 4.*exp(-(a+h)*(a+h)/2.);
  // and the last one
  sum = sum + exp(-b*b/2.);
  // loop over terms 2 up to N-1
  for(int i=1;i<N/2;i++)
  {
    s = a + 2*i*h;
    sum = sum + 2.*exp(-s*s/2.);
    s = s + h;
    sum = sum + 4.*exp(-s*s/2.);
  }
  // complete the integral
  sum = 0.5 + h*sum/3./sqrt(8.*atan(1.));
  // return result
  return sum;
}

## Tasks

- Test your code by outputting some values of $N(x)$ for $x$ in the range $[-10,10].$
- Try varying the number of steps in the integration, what happens for very small or very large numbers?
- Try using a polynomial approximation to $N(x)$ (less than single precision!): $$ N(x)= 1-N'(x)(a_1 k+a_2 k^2+a_3 k^3), \quad x\geq 0,$$ $$N(x)=1-N(-x),\quad x<0.$$ Here $k=\frac{1}{1+\gamma x}$, $\gamma=0.33267$, $a_1=0.43618$,
$a_2=-0.12017$, and $a_3=0.93730$.

Check out the value at a few points:

In [3]:
{
    cout << normalDistribution_int(0,128) << endl;
    cout << normalDistribution_int(1,128) << endl;
    cout << normalDistribution_int(2,128) << endl;
    cout << normalDistribution_int(3,128) << endl;
    cout << normalDistribution_int(4,128) << endl;
    cout << normalDistribution_int(5,128) << endl;
}

0.5
0.841345
0.97725
0.99865
0.999968
1


Now run the function with increasing values of $n$, notice that the value stops changing when the value of $n$ gets large, seeming that the solution has converged.

In [4]:
for(int n=2;n<=5000;n*=2)
    cout << n << " " << normalDistribution_int(1,n) << endl;

2 0.841529
4 0.841355
8 0.841345
16 0.841345
32 0.841345
64 0.841345
128 0.841345
256 0.841345
512 0.841345
1024 0.841345
2048 0.841345
4096 0.841345


Now we implement the polynomial function approximation as given:

In [5]:
double normalDistribution_poly(double x)
{
    double gmm=0.33267;
    double a1=0.43618;
    double a2=-0.12017;
    double a3=0.93730;
    double k=1/(1+gmm*x);
    double Ndash = exp(-x*x/2.)/sqrt(8.*atan(1.));
    if(x>=0.)return 1.-Ndash*((a3*k+a2)*k+a1)*k;
    else return Ndash*((a3*k+a2)*k+a1)*k;
}

Notice that the solution is similar but there is clearly an error of the order $10^{-4}$ in places.

In [6]:
{
    cout << normalDistribution_poly(0) << endl;
    cout << normalDistribution_poly(1) << endl;
    cout << normalDistribution_poly(2) << endl;
    cout << normalDistribution_poly(3) << endl;
    cout << normalDistribution_poly(4) << endl;
    cout << normalDistribution_poly(5) << endl;
}

0.500002
0.841352
0.977241
0.998645
0.999968
1


## Code from Demo 1.2

In [7]:
#include <iomanip>
#include <chrono>

In [8]:
{
  int N=100000;
  // get start time
  auto start = std::chrono::steady_clock::now(); 
  // code in here is timed
  double sum=0.;
  for(int i=0;i<N;i++)
  {
    for(int x=-5;x<=5;x++)
      sum = sum + normalDistribution_poly(x);
  }
  cout << sum << endl;
  // get finish time
  auto finish = std::chrono::steady_clock::now();
  // convert into real time in seconds
  auto elapsed = std::chrono::duration_cast<std::chrono::duration<double> >(finish - start);
  // output values
  cout << " Total time elapsed for "; 
  cout << 11.*N << " calculations is " << elapsed.count() << endl;
}

5.26988e+10
 Total time elapsed for 1.1e+06 calculations is 0.0493652


## Tasks

- Test the efficiency of your calculation for $N(x)$ against the polynomial approximation -- which method is the best one?
- Try using the standard library error function `erfc`, how does this compare to other methods?
- Go through the solutions for the Black-Scholes equation -- note solution codes are provided for all options.

## Solutions


Now try the integral approximation `normalDistibution_int` with $n=128$ steps in the integration and run it $N=100000$ times. Choosing higher values than this may mean waiting a long time!

In [9]:
{
  int N=100000;
  // get start time
  auto start = std::chrono::steady_clock::now(); 
  // code in here is timed
  double sum=0.;
  for(int i=0;i<N;i++)
  {
    for(int x=-5;x<=5;x++)
      sum = sum + normalDistribution_int(x,128);
  }
  cout << sum << endl;
  // get finish time
  auto finish = std::chrono::steady_clock::now();
  // convert into real time in seconds
  auto elapsed = std::chrono::duration_cast<std::chrono::duration<double> >(finish - start);
  // output values
  cout << " Total time elapsed for "; 
  cout << 11.*N << " calculations is " << elapsed.count() << endl;
}

550000
 Total time elapsed for 1.1e+06 calculations is 1.80702


The results for the integration are more accurate, but take longer to calculate. So clearly we need to measure the trade off between accuracy and speed.

Now try the standard library function

In [10]:
double normalDistribution_std(double x)
{
    return 0.5*erfc(-x/sqrt(2.));
}

Check we've done the transformation correctly

In [11]:
{
    cout << normalDistribution_std(0) << endl;
    cout << normalDistribution_std(1) << endl;
    cout << normalDistribution_std(2) << endl;
    cout << normalDistribution_std(3) << endl;
    cout << normalDistribution_std(4) << endl;
    cout << normalDistribution_std(5) << endl;
}

0.5
0.841345
0.97725
0.99865
0.999968
1


In [12]:
{
  int N=100000;
  // get start time
  auto start = std::chrono::steady_clock::now(); 
  // code in here is timed
  double sum=0.;
  for(int i=0;i<N;i++)
  {
    for(int x=-5;x<=5;x++)
      sum = sum + normalDistribution_std(x);
  }
  cout << sum << endl;
  // get finish time
  auto finish = std::chrono::steady_clock::now();
  // convert into real time in seconds
  auto elapsed = std::chrono::duration_cast<std::chrono::duration<double> >(finish - start);
  // output values
  cout << " Total time elapsed for "; 
  cout << 11.*N << " calculations is " << elapsed.count() << endl;
}

550000
 Total time elapsed for 1.1e+06 calculations is 0.0486397


Ok, so this is just as fast as the polynomial approximation but easier to write and accurate to machine precision -- we have a winner!